In [7]:
import pandas as pd

df = pd.read_csv('data\dog_adopt_eda_data.csv')

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Admin\AppData\Local\Temp\ipykernel_6256\2963335214.py:3: SyntaxWarning: invalid escape sequence '\d'
  df = pd.read_csv('data\dog_adopt_eda_data.csv')


In [8]:
# 성격, 건강, 색 : 글로 작성해도 임베딩하여 군집화 해서 근처의 것끼리 묶어서 판별

from sentence_transformers import SentenceTransformer
import torch 
from sklearn.cluster import KMeans
import pickle

def encode_cluster_df(column, scaler_path):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model = SentenceTransformer('snunlp/KR-SBERT-V40K-klueNLI-augSTS')
    embeddings = model.encode(df[column].tolist(), device=device, convert_to_tensor=True)

    embeddings_np = embeddings.cpu().numpy()

    kmeans = KMeans(n_clusters=5, random_state=42)
    clusters = kmeans.fit_predict(embeddings_np)
    with open(scaler_path, 'wb') as f:
        pickle.dump(kmeans, f)

    df[column] = clusters
    
encode_cluster_df('건강', 'scaler/health_kmeans')
encode_cluster_df('성격', 'scaler/character_kmeans')
encode_cluster_df('색', 'scaler/color_kmeans')

c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\transformers\tokenization_utils_base.py

TypeError: 'int' object is not subscriptable

In [11]:
from sklearn.preprocessing import LabelEncoder
import pickle

def label_encode(column, scaler_path):
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    df[column].value_counts()

    with open(scaler_path, 'wb') as f:
        pickle.dump(le, f)

label_encode('칩등록여부', 'scaler\chip_labelencoder.pkl')
label_encode('보유물건', 'scaler\property_labelencoder.pkl')
label_encode('중성화유무', 'scaler\property_labelencoder.pkl')
label_encode('성별', 'scaler\sex_labelencoder.pkl')
label_encode('품종', 'scaler/breed_labelencoder.pkl')

<>:12: SyntaxWarning: invalid escape sequence '\c'
<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\c'
<>:13: SyntaxWarning: invalid escape sequence '\p'
<>:14: SyntaxWarning: invalid escape sequence '\p'
<>:15: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Admin\AppData\Local\Temp\ipykernel_6256\2661194786.py:12: SyntaxWarning: invalid escape sequence '\c'
  label_encode('칩등록여부', 'scaler\chip_labelencoder.pkl')
C:\Users\Admin\AppData\Local\Temp\ipykernel_6256\2661194786.py:13: SyntaxWarning: invalid escape sequence '\p'
  label_encode('보유물건', 'scaler\property_labelencoder.pkl')
C:\Users\Admin\AppData\Local\Temp\ipykernel_6256\2661194786.py:14: SyntaxWarning: invalid escape sequence '\p'
  label_encode('중성화유무', 'scaler\property_labelencoder.pkl')
C:\Users\Admin\AppData\Local\Temp\ipykernel_6256\2661194786.py:15: SyntaxWarning: in

In [18]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['상태'], axis=1)
y = df[['상태']]
print(X.shape)  # 피처 데이터의 크기 출력
print(y.shape)  # 타겟 데이터의 크기 출력
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

(2496, 10)
(2496, 1)


In [19]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()

X_train[['나이', '무게(Kg)']] = scaler.fit_transform(X_train[['나이', '무게(Kg)']])
X_test[['나이', '무게(Kg)']] = scaler.transform(X_test[['나이', '무게(Kg)']])

with open('scaler/minmax_scaler_age_weight.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [20]:
import category_encoders as ce
import pickle

target_encoder = ce.TargetEncoder(cols=['품종'])
X_encoded = target_encoder.fit_transform(X_train, y_train)
X_test_encoded = target_encoder.transform(X_test)

with open('scaler/breed_targetencoder.pkl', 'wb') as f:
    pickle.dump(target_encoder, f)

c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packages\category_encoders\ordinal.py:198: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X[column] = X[column].astype("object").fillna(np.nan).map(col_mapping)
c:\Users\Admin\Desktop\metaverse\project\dog_adopt_probability\.venv\Lib\site-packag

In [22]:
X_encoded.to_csv('dog_data/X_train_data.csv', encoding='utf-8-sig', index=False)
X_test_encoded.to_csv('dog_data/X_test_data.csv', encoding='utf-8-sig', index=False)
y_train.to_csv('dog_data/y_train_data.csv', encoding='utf-8-sig', index=False)
y_test.to_csv('dog_data/y_test_data.csv', encoding='utf-8-sig', index=False)